In [16]:
from pyeda.inter import *
import yaml
import re
import pickle
from collections import Counter

In [89]:
class AutoGraderHelpers:
    # some helpers
    @staticmethod
    def checkAllStr(l):
        assert type(l) is list
        return all(type(e) is str for e in l)
    
    @staticmethod
    def checkAllList(l):
        assert type(l) is list
        return all(type(e) is list for e in l)
    
    # testin and answer are strings or list of stirngs, and are matched element by element
    @classmethod
    def matchElementInList(cls,answer,testin,cond=None,matchtype='exact'):
        assert matchtype in ['int','float','exact','nocase']
        assert cond in [None,'all','any']
        testin = [e.split() for e in testin.strip().split('\n')]
        assert type(answer) is str or cls.checkAllStr(answer)
        if type(answer) is str:
            answer = [answer]
        if matchtype=='int':
            flags = [int(e1)==int(e2) for e1,e2 in zip(answer,testin)]
        elif matchtype=='float':
            flags = [float(e1)==float(e2) for e1,e2 in zip(answer,testin)]
        elif matchtype=='nocase':
            flags = [e1.lower()==e2.lower() for e1,e2 in zip(answer,testin)]
        else:
            flags = [e1==e2 for e1,e2 in zip(answer,testin)]
        if cond=='all':
            return all(flags)
        elif cond=='any':
            return any(flags)
        else:
            return flags
        
    # each line of answer is expected to be a singleton or a list, and 
    # supposed to match to a corresponding singleton or a list of the same length
    @classmethod
    def matchList(cls,answer,testin,cond=None,ordered=False):
        assert cond in [None,'all','any']
        testin = [e.split() for e in testin.strip().split('\n')]
        assert cls.checkAllStr(answer) or (cls.checkAllList(answer) and all(cls.checkAllStr(e) for e in answer))
        if cls.checkAllStr(answer):
            answer = [answer]
        if len(answer)!=len(testin):
            return False
        if ordered:
            flags = [e1==e2 for e1,e2 in zip(answer,testin)]
        else:
            flags = [sorted(e1)==sorted(e2)for e1,e2 in zip(answer,testin)]
        if cond=='all':
            return all(flags)
        elif cond=='any':
            return any(flags)
        else:
            return flags
    
    #
    # answer: "D101010110101DDD"
    @staticmethod
    def boolEquivalentTruthTable(answer,testin):
        testin = testin.strip().split('\n')
        return False
    
    #
    # answer: [D110, 0110, 1DDD, 1001]
    @staticmethod
    def boolEquivalentKMap(answer,testin):
        testin = testin.strip().split('\n')
        return False
    
        
    # boolean expression string or a list of boolean expression strings
    @staticmethod
    def boolEquivalentExpression(vars,answer,testin,form=None,ops=None):
        testin = testin.strip().split('\n')
        assert type(answer) is str or type(answer) is list
        if type(answer) is str:
            answer = [answer]
        assert len(testin)==len(answer)
        assert form in [None, 'pos', 'sop', 'DNF', 'CNF']
        for _vn in vars:
            exec("{0}=exprvar('{0}')".format(_vn))
        _results=[]
        for _e1,_e2 in zip(answer,testin):
            _results.append(expr(_e1).equivalent(_e2))
        return _results
    
    @staticmethod
    def logisimCombinationalTester(self,answer,testin,config,logisim=None):
        pass

In [102]:
def fixBooleanOperators(e):
	a = ""
	for i in range(len(e)):
		if e[i]=='!':
			a=a+'~'
		elif e[i]=='*':
			a=a+'&'
		elif e[i]=='+':
			a=a+'|'
		else:
			a=a+e[i]
	return a

In [110]:
' 32 32 '.split()

['32', '32']

In [18]:
class YAMLHelpers:
    @staticmethod
    def extractHeader(allcontent):
        all_content = allcontent.strip().rstrip()
        yaml_header = {}
        if all_content[:3]=="---":
            i = all_content[4:].find("\n---")
            if i!=-1:
                try:
                    #import pdb; pdb.set_trace()
                    yaml_header = yaml.full_load(all_content[0:i+5])
                    all_content = all_content[i+9:].strip()
                except yaml.scanner.ScannerError as e:
                    raise SystemExit("Error: YAML Header has error: {0}".format(e))
                except:
                    raise SystemExit("Error: YAML Header has error")
        return (yaml_header,all_content) 

In [29]:
DEFAULT_GRADINGINFO_FILE = '/Users/mbs/Dropbox/Documents/Courses/eem16/2020s/autograders/pset1/grading_info.pkl'
DEFAULT_ANSWER_FILE = '/Users/mbs/Dropbox/Documents/Courses/eem16/2020s/autograders/pset1/samples/submission_35409712/answers.txt'

class AutoGrader():
    def __init__(self, gradingfilename=DEFAULT_GRADINGINFO_FILE, answerfilename=DEFAULT_ANSWER_FILE):
        self.grading_info = {(e['probnum'],e.get('subprobnum',None),e.get('version',None)): {
            'answer_key':e.get('answer_key',None), 
            'probpoints':e.get('probpoints',None)} for e in list(pickle.load(open(gradingfilename,'rb')).values())[0]}
        self.answers = {(e['probnum'],e.get('subprobnum',None),e.get('version',None)): e.get('text',None)
                        for e in self.readAnswerFile(answerfilename)}

    @staticmethod
    def readAnswerFile(answerfilename):
        with open(answerfilename,'r') as f:
            content = f.read()
        header,body = YAMLHelpers.extractHeader(content)
        body = body.split('---\n')
        matcher = re.compile(r'(#(.*?)\n)+')
        answers = []
        for answer_text in body:
            if answer_text[0:8]!="# Answer":
                continue
            answer = {}
            m = re.search(r'# Answer\s+(\d+)(.(\d+))?(\s+\[ver(\d+)\])?\s*\n',answer_text)
            answer['probnum'] = m[1]
            if m[3]: answer['subprobnum'] = m[3]
            if m[5]: answer['version'] = m[5]
            answer['text'] = []
            answer_text_parts = matcher.sub('#\n',answer_text).split('#\n')
            for answer_text_part in answer_text_parts:
                if answer_text_part=="":
                    continue
                answer['text'].append(list(filter(lambda x: x!='',answer_text_part.split('\n'))))
            answers.append(answer)
        return answers
    
    @staticmethod
    def makePrettyProblemNumber(p):
        r = "Problem {0}".format(p[0])
        if len(p)>1 and p[1]:
            r += ", Subproblem {0}".format(p[1])
        if len(p)>2 and p[2]:
            r += ", Version {0}".format(p[2])
        return r
        
    def doGrading(self,sanityCheckOnly=True):
        # check that answer sheet only has questions that exist in the assignment
        if not set(self.answers.keys()).issubset(set(self.grading_info.keys())):
            m = "Submission has questions not in the assignment: {0}".format(
                set(self.answers.keys())-set(self.grading_info.keys()))
            print(m)
            
        # check that questions are answered only once
        C = Counter([(e[0],e[1]) for e in self.answers.keys()])
        results = {e:[] for e in self.grading_info.keys()}
        errmsg = ""
        for problem in C:
            if C[problem]>1:
                m = "Multiple answers for {0}".format(self.makePrettyProblemNumber(problem))
                print(m)
                results[problem] = {'error':[m]}
            else:
                results[problem] = {'error':[]}
        print(C)
        # now check format of anwers for every question
        for problem,answers in self.answers.items():
            if results[problem]!=[]:
                print("\nSkipping {0} due to prior error '{1}''".format(self.makePrettyProblemNumber(problem)),results[problem])
                continue
            else:
                print("\nAnalyzing {0}".format(self.makePrettyProblemNumber(problem)))
            #
            keys = self.grading_info.get(problem,{}).get('answer_key',[])
            if (len(answers)!=len(keys)):
                m = "Incorrect # of parts in answer for {0}: expected {1} parts, got {2} parts.".format(
                    self.makePrettyProblemNumber(problem),len(keys),len(answers))
                results[problem].append(m)
                print("\nSkipping {0} due to error '{1}''".format(self.makePrettyProblemNumber(problem)),m)
                print(m+"\n"+self.makePrettyProblemNumber(problem)+"\n"+str(answers)+"\n"+str(keys))
                continue
            #
            for partnum, (apart, kpart) in enumerate(zip(answers,keys)):
                print("\nTesting part {0} of {1}".format(partnum,self.makePrettyProblemNumber(problem)))
                if (kpart['type']=='int'):
                    a = [e.replace(',',' ').split() for e in apart]
                    if type(kpart['answer'])==list:
                        k = [e.split() if type(e)==str else e for e in eval(kpart['answer'])]
                    elif type(kpart['answer'])==str:
                        k = eval(kpart['answer'])
                    else:
                        m = "Configuration Error: answer in {0} should be string or list".format(kpart)
                        results[problem].append(m)
                        print("\nSkipping part {0} of {1} due to error '{2}''".format(partnum,self.makePrettyProblemNumber(problem)),m)
                        print(m+"\n"+self.makePrettyProblemNumber(problem)+"\n"+str(answers)+"\n"+str(keys))
                        continue
                    # check that number of lines in answer is the same as in key
                    if len(a)!=len(k):
                        m = "Incorrect # of lines in answer for part # {0} of {1}: expected {2} lines, got {3} lines.".format(
                            partnum,self.makePrettyProblemNumber(problem),len(k),len(a))
                        results[problem].append(m)
                        print("\nSkipping part {0} of {1} due to error '{2}''".format(partnum,self.makePrettyProblemNumber(problem)),m)
                        print(m+"\n"+self.makePrettyProblemNumber(problem)+"\n"+str(answers)+"\n"+str(keys))
                        continue
                    for i,e in enumerate(k):
                        print("Line #",i+1,": testing",e,"against",a[i])
    
                    continue
                elif (kpart['type']=='str'):
                    pass
                elif (kpart['type']=='float'):
                    pass
                elif (kpart['type']=='kmap'):
                    pass
                elif (kpart['type']=='truthtable'):
                    pass
                elif (kpart['type']=='file'):
                    pass
                elif (kpart['type']=='other'):
                    pass
            return results
        return results

In [40]:
pickle.load(open(DEFAULT_GRADINGINFO_FILE,'rb'))[('605125539','ZHANG','JACOB')]

[{'probnum': '1',
  'probpoints': '5',
  'probfile': 'digital_abstraction/color_wheel.md',
  'answer_key': [{'type': 'int', 'answer': '[4, [5, 6]]'},
   {'type': 'other',
    'helper': 'graderColorWheel',
    'format': '^([a-z\\-]+:\\s*[01\\s]*(\\n)?){12}$',
    'answer': ''}]},
 {'probnum': '2',
  'probpoints': '5',
  'probfile': 'digital_abstraction/genetic_code_ankur.md',
  'answer_key': [{'type': 'int', 'answer': '2'},
   {'type': 'int', 'answer': '5'},
   {'type': 'int', 'answer': '3'},
   {'type': 'int', 'answer': '3'},
   {'type': 'int', 'answer': '27'},
   {'type': 'int', 'answer': '17'},
   {'type': 'int', 'answer': '12'}]},
 {'probnum': '3',
  'probpoints': '3',
  'probfile': 'boolean_algebra/circuit_to_dual_1.md',
  'subprobnum': '1',
  'answer_key': [{'type': 'other',
    'answer': '(a | c | d) & (~a | ~b | ~c) & (~a | ~b | ~d) & (b & c & d)',
    'helper': {'func': 'boolEquivalentExpression',
     'vars': ['a', 'b', 'c', 'd']}}]},
 {'probnum': '3',
  'probpoints': '3',
  '

In [46]:
pickle.load(open(DEFAULT_GRADINGINFO_FILE,'rb'))

{('605391825',
  'ANDERSEN',
  'DESMOND'): [{'probnum': '1',
   'probpoints': '5',
   'probfile': 'digital_abstraction/color_wheel.md',
   'answer_key': [{'type': 'int', 'answer': '[4, [5, 6]]'},
    {'type': 'other',
     'helper': 'graderColorWheel',
     'format': '^([a-z\\-]+:\\s*[01\\s]*(\\n)?){12}$',
     'answer': ''}]}, {'probnum': '2',
   'probpoints': '5',
   'probfile': 'digital_abstraction/genetic_code_ankur.md',
   'answer_key': [{'type': 'int', 'answer': '2'},
    {'type': 'int', 'answer': '5'},
    {'type': 'int', 'answer': '3'},
    {'type': 'int', 'answer': '3'},
    {'type': 'int', 'answer': '27'},
    {'type': 'int', 'answer': '17'},
    {'type': 'int', 'answer': '12'}]}, {'probnum': '3',
   'probpoints': '3',
   'probfile': 'boolean_algebra/circuit_to_dual_1.md',
   'subprobnum': '1',
   'answer_key': [{'type': 'other',
     'answer': '(a | c | d) & (~a | ~b | ~c) & (~a | ~b | ~d) & (b & c & d)',
     'helper': {'func': 'boolEquivalentExpression',
      'vars': ['a'

In [41]:
AutoGrader.readAnswerFile(DEFAULT_ANSWER_FILE)

[{'probnum': '1',
  'text': [['4'],
   ['red: 0000',
    'red-orange: 0001 ',
    'orange: 0011',
    'yellow-orange:  0010',
    'yellow: 0110',
    'yellow-green: 0111 ',
    'green: 0101',
    'blue-green: 0100 ',
    'blue: 1100',
    'blue-violet: 1110 ',
    'violet: 1010',
    'red-violet: 1000']]},
 {'probnum': '2',
  'text': [['2'], ['5'], ['3'], ['3'], ['27'], ['17'], ['12']]},
 {'probnum': '3',
  'subprobnum': '1',
  'text': [['((a&b)|(c|d))&(~(a&b)|~(c|d))']]},
 {'probnum': '3', 'subprobnum': '2', 'text': [['(~a&~b&c)|(~a&b&c)']]},
 {'probnum': '3', 'subprobnum': '3', 'text': [['32 32'], ['3_3.pdf']]},
 {'probnum': '3',
  'subprobnum': '4',
  'text': [['1',
    'DNF',
    '(~a&~b&~c)|(a&~b&~c)|(a&~b&c)|(a&b&~c)',
    '(~a&~b&~c)|(a&~b&~c)|(a&~b&c)|(a&b&~c)']]},
 {'probnum': '4',
  'text': [['0 0 0 0 D',
    '0 0 0 1 1',
    '0 0 1 0 0',
    '0 0 1 1 1',
    '0 1 0 0 0',
    '0 1 0 1 1',
    '0 1 1 0 0',
    '0 1 1 1 1',
    '1 0 0 0 1',
    '1 0 0 1 0',
    '1 0 1 0 1',
   

In [42]:
ag = AutoGrader()

In [43]:
ag.grading_info.keys()

dict_keys([('1', None, None), ('2', None, None), ('3', '1', None), ('3', '2', None), ('3', '3', None), ('3', '4', None), ('4', None, None), ('5', None, '1'), ('6', None, None), ('7', None, None), ('8', None, None), ('9', None, None)])

In [44]:
ag.answers.keys()

dict_keys([('1', None, None), ('2', None, None), ('3', '1', None), ('3', '2', None), ('3', '3', None), ('3', '4', None), ('4', None, None), ('5', None, '2'), ('6', None, None), ('7', None, None), ('8', None, None), ('9', None, None)])

In [45]:
ag.doGrading()

Submission has questions not in the assignment: {('5', None, '2')}
Counter({('1', None): 1, ('2', None): 1, ('3', '1'): 1, ('3', '2'): 1, ('3', '3'): 1, ('3', '4'): 1, ('4', None): 1, ('5', None): 1, ('6', None): 1, ('7', None): 1, ('8', None): 1, ('9', None): 1})

Analyzing Problem 1

Testing part 0 of Problem 1


IndexError: Replacement index 2 out of range for positional args tuple

In [149]:
[e.split() if type(e)==str else e for e in eval(ag.doGrading()[1]['answer'])]

Multiple answers for Problem 5

Analyzing Problem 1
[['7', '8']]
Incorrect # of lines in answer for part # 0 of Problem 1: expected 2 lines, got 1 lines.
Problem 1
[['7, 8'], ['red: ', 'red-orange: ', 'orange: ', 'yellow-orange: ', 'yellow: ', 'yellow-green: ', 'green: ', 'blue-green: ', 'blue: ', 'blue-violet: ', 'violet: ', 'red-violet: ']]
[{'type': 'int', 'answer': '[4, [5, 6]]'}, {'type': 'other', 'helper': 'graderColorWheel', 'format': '^([a-z\\-]+:\\s*[01\\s]*(\\n)?){12}$', 'answer': ''}]
Line # 1 : testing 4 against ['7', '8']


IndexError: list index out of range

In [97]:
'5, 6, 7'.replace(',',' ').split()

['5', '6', '7']

In [59]:
a.processAssignment()
a.saveGradingInfo()
print("DONE!")

NameError: name 'a' is not defined

In [474]:
eval('5')

5

In [470]:
ag.grading_info[('1',None,None)]['answer_key'][0]

{'type': 'other',
 'answer': '[[2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, 1, 4, 5]]',
 'helper': {'func': 'matchList', 'cond': 'all'}}

In [463]:
' '.join(ag.answers[('1',None,None)][0])

'5 6 7'

In [406]:
list(ag.answers.keys())

[('1', None, None),
 ('2', None, None),
 ('3', '1', None),
 ('3', '2', None),
 ('3', '3', None),
 ('3', '4', None),
 ('4', None, None),
 ('5', None, '1'),
 ('5', None, '2'),
 ('6', None, None),
 ('7', None, None),
 ('8', None, None),
 ('9', None, None)]

In [297]:
list(ag.grading_info.keys())

[('1', None, None),
 ('2', None, None),
 ('3', '1', None),
 ('3', '2', None),
 ('3', '3', None),
 ('3', '4', None),
 ('4', None, None),
 ('5', None, '1'),
 ('5', None, '2'),
 ('6', None, None),
 ('7', None, None),
 ('8', None, None),
 ('9', None, None)]

{('1', None): {'version': None,
  'answer_key': [{'type': 'int', 'answer': '4'},
   {'type': 'other',
    'helper': 'graderColorWheel',
    'format': '.*:\\s+[01]*',
    'answer': ''}],
  'probpoints': '5'},
 ('2', None): {'version': None,
  'answer_key': [{'type': 'int', 'answer': '2'},
   {'type': 'int', 'answer': '5'},
   {'type': 'int', 'answer': '3'},
   {'type': 'int', 'answer': '3'},
   {'type': 'int', 'answer': '27'},
   {'type': 'int', 'answer': '17'},
   {'type': 'int', 'answer': '12'}],
  'probpoints': '5'},
 ('3', '1'): {'version': None,
  'answer_key': [{'type': 'int', 'answer': '2'},
   {'type': 'int', 'answer': '5'},
   {'type': 'int', 'answer': '3'},
   {'type': 'int', 'answer': '3'},
   {'type': 'int', 'answer': '27'},
   {'type': 'int', 'answer': '17'},
   {'type': 'int', 'answer': '12'}],
  'probpoints': '0'},
 ('3', '2'): {'version': None,
  'answer_key': [{'type': 'int', 'answer': '2'},
   {'type': 'int', 'answer': '5'},
   {'type': 'int', 'answer': '3'},
   {'type

In [255]:
ag.answers

[{'probnum': '1',
  'text': [[],
   ['red: ',
    'red-orange: ',
    'orange: ',
    'yellow-orange: ',
    'yellow: ',
    'yellow-green: ',
    'green: ',
    'blue-green: ',
    'blue: ',
    'blue-violet: ',
    'violet: ',
    'red-violet: ']]},
 {'probnum': '2', 'text': [[], [], [], [], [], [], []]},
 {'probnum': '3', 'subprobnum': '1', 'text': [[]]},
 {'probnum': '3', 'subprobnum': '2', 'text': [[]]},
 {'probnum': '3', 'subprobnum': '3', 'text': [[], ['3_ver3.pdf']]},
 {'probnum': '3', 'subprobnum': '4', 'text': [[]]},
 {'probnum': '4', 'text': [[], [], [], ['assets_4/month31.circ']]},
 {'probnum': '5',
  'version': '1',
  'text': [[], [], [], ['assets_5_ver1/boolean_counting.circ']]},
 {'probnum': '5',
  'version': '2',
  'text': [[], [], [], [], ['assets_5_ver2/ternary_operator.circ']]},
 {'probnum': '6', 'text': [[], ['assets_6/thermometer_encoder.circ']]},
 {'probnum': '7', 'text': [[], [], []]},
 {'probnum': '8', 'text': [['8.pdf']]},
 {'probnum': '9', 'text': [[' 9.pdf']]

In [188]:
b.split('---\n')[3]

'# Answer 3.1\n#\n# Write the boolean algebra expression as specified. Use only operators &, |, and ~. \n# (only write the expression in terms of a, b, c, and d - do not include "x =")\n\n'

In [169]:
re.search(r'# Answer\s+(\d+)(.(\d+))?\n',b.split('---\n')[3])[3]

'1'

In [170]:
p = re.compile(r'(#(.*?)\n)+')
p.sub('#\n',b.split('---\n')[3]).split('#\n')

['', '\n']

In [55]:
g = AutoGraderHelpers()

In [56]:
g.matchList(answer=[['0','1','4','5'],['2','4']],testin="1 0 4 5\n2 3",cond='any')

True

In [57]:
g.boolEquivalentExpression(vars=['a','b','c'],testin="a&b",answer="a&b")

[True]

In [58]:
exprvar('c')

c

In [ ]:
y = []
for e in map(exprvar,x):
    y.append(e)

In [ ]:
z = 'a & b | ~b'

In [ ]:
exec("{0}=y[0]".format(x[0]))
exec("{0}=y[1]".format(x[1]))

In [ ]:
a & b | ~b

In [ ]:
expr('a&b').equivalent(expr('a&b'))

In [ ]:
for x,y in zip([1,2],[3,4]):
    print(x,y)

In [48]:
x = ['red: 0000', 'red-orange: 0001 ', 'orange: 0011', 'yellow-orange:  0010', 'yellow: 0110', 'yellow-green: 0111 ', 'green: 0101', 'blue-green: 0100 ', 'blue: 1100', 'blue-violet: 1110 ', 'violet: 1010', 'red-violet: 1000']

In [52]:
set({e.split(':')[0].strip():e.split(':')[1].strip() for e in x}.keys())

{'blue',
 'blue-green',
 'blue-violet',
 'green',
 'orange',
 'red',
 'red-orange',
 'red-violet',
 'violet',
 'yellow',
 'yellow-green',
 'yellow-orange'}

In [54]:
set({e.split(':')[0].strip():e.split(':')[1].strip() for e in x}.keys()) == set(['blue',
 'blue-green',
 'blue-violet',
 'green',
 'orange',
 'red',
 'red-orange',
 'red-violet',
 'violet',
 'yellow',
 'yellow-green',
 'yellow-orange'])

True

In [85]:
def getKeySetEqual(x,s):
    return set({e.split(':')[0].strip():e.split(':')[1].strip() for e in x}.keys())

def checkColorWheelCoding(x):
    colors = [
        'red', 'red-orange', 'orange', 'yellow-orange', 'yellow', 'yellow-green', 'green', 
        'blue-green', 'blue', 'blue-violet', 'violet', 'red-violet']
    answer = {e.split(':')[0].strip():e.split(':')[1].strip() for e in x}
    if set(answer.keys()) != set(colors):
        print("1")
        return False
    if any([len(e)!=4 and re.search(r"[01]{4}", e) for e in answer.values()]):
        print("2")
        return False
    current = answer[colors[-1]]
    used = []
    for c in colors:
        previous = current
        current = answer[c]
        if current in used:
            print("3")
            return False
        used.append(current)
        diff = False
        for i in range(4):
            if previous[i]!=current[i]:
                if diff:
                    return False
                else:
                    diff = True
    return True

In [118]:
x = ['0 0 0 0 D', '0 0 0 1 1', '0 0 1 0 0', '0 0 1 1 1', '0 1 0 0 0', '0 1 0 1 1', '0 1 1 0 0', '0 1 1 1 1', '1 0 0 0 1', '1 0 0 1 0', '1 0 1 0 1', '1 0 1 1 0', '1 1 0 0 1', '1 1 0 1 D', '1 1 1 0 D', '1 1 1 1 D']

In [121]:
{int(''.join(e.split())[:4],base=2):''.join(e.split())[4] for e in x}

{0: 'D',
 1: '1',
 2: '0',
 3: '1',
 4: '0',
 5: '1',
 6: '0',
 7: '1',
 8: '1',
 9: '0',
 10: '1',
 11: '0',
 12: '1',
 13: 'D',
 14: 'D',
 15: 'D'}

In [117]:
AutoGraderHelpers.boolEquivalentExpression(['M3','M2','M1','M0'],'(M3 | M0) & (~M3 | ~M0)','(~M3&M0)|(M3&~M0)')

[True]

In [132]:
''.join('in2 | in1 | in0)'.split('=')[-1].split())

'in2|in1|in0)'

In [134]:
['4'][0].strip()

'4'